# Labor 3

A párhuzamos algoritmusok célja a [teljesítmény növelése](https://courses.cs.washington.edu/courses/csep524/07sp/poppChaper1.pdf). Ezt azzal érik el, hogy a számításokat egyszerre több számítógépen vagy számítási egységen végzik el, így csökkentve a számítási időt, vagy lehetővé téve pontosabb eredmények elérését egy adott időegység alatt.


# Teljesítménymértékek

A teljesítménynövekedés számszerű kifejezése érdekében áttekintjük, hogy milyen módszerek állnak rendelkezésre.

## Gyorsítás (speedup)

Az időarány, amelyben a párhuzamos algoritmus végrehajtása gyorsabb, mint a szekvenciális algoritmusé. A gyorsítás értéke az eredeti, szekvenciális idő és az új idő hányadosa.

Jelölje $T(n,p)$ azt a függvényt, ami megadja, hogy egy párhuzamos program egy $n$ méretű feladatot mennyi idő alatt old meg $p$ processzor használatával.

Adott $n$-méretű feladatra a $T(n,p)$ párhuzamos $p$ processzoron futó program reális  **gyorsítását (gyorsítási faktort)**

$S(n,p)=\frac{T^*(n)}{T(n,p)}$

adja meg, ahol $T^*(n)$ a leggyorsabb ismert soros algoritmus futási ideje.



A környezettől függően a gyorsítás vonatkozhat $p$ folyamat vagy $p$ szál alkalmazására is.

$T^*(n)$ nem mindig ismert, vagy nem implementálható, ezért gyakran a relatív **_gyorsítását_**  használjuk, ahol a párhuzamos program egy processzoros futási idejét vesszük figyelembe:

$S(n,p)=\frac{T(n,1)}{T(n,p)}$


## Hatékonyság (efficiency)

Másik fontos metrika a **_párhuzamos hatékonyság_** ami kifejezi a gyorsítás arányát a felhasznált számítási erőforrásokhoz képest. Az hatékonyság értéke a gyorsulás és a felhasznált erőforrások (például a processzorok vagy a magok száma) hányadosa:

$E(n,p)=\frac{S(n,p)}{p}=\frac{T^*(n)}{p*T(n,p)}$

Ez a mutató betekintést nyújt a processzorok kihasználtságára, ideális esetben ez az érték 1 (lineáris gyorsulás).

Egy párhuzamos programot akkor nevezünk **_skálázhatónak_**, ha a párhuzamos hatékonyság fenntartható. Skálázhatóság (scalability): a rendszer képessége, hogy a számítási feladatokat nagyobb erőforrásokkal bővítve hatékonyan kezelje.



# Időmérésa CUDA események segítségével

A CUDA esemény ([CUDA event](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html#group__CUDART__EVENT)) egy szinkronizációs jel, amely lehetővé teszi a programozó számára, hogy monitorizálja a CUDA munkafolyamatokat, szinkronizálja a CUDA adatfolyamokat (CUDA streams) és lehetőve teszi a [CUDA kernelek pontos időmérését](https://developer.nvidia.com/blog/how-implement-performance-metrics-cuda-cc/).


A `cudaEvent` egy CUDA API függvény, amely lehetővé teszi a programozók számára, hogy mérjék a CUDA műveletek végrehajtási idejét. A `cudaEvent` funkciók segítségével időbélyegeket állíthatunk be, majd ezen bélyegek között eltelt időt mérhetjük.

A következő lépésekkel használjuk a `cudaEvent`-et időmérésre:

1. Létre kell hozni két `cudaEvent` objektumot, amelyek közül az egyiket a művelet kezdeteként, a másikat pedig a művelet befejezéseként jelöljük meg. Például:
  ```cpp
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  ```
2. A [`cudaEventRecord`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1gf4fcb74343aa689f4159791967868446) függvény segítségével be kell állítanunk a start és stop bélyegeket a művelet elkezdése elött és a befejezésnél. Például:
  ```cpp
  cudaEventRecord(start, 0);
  // CUDA műveletek végrehajtása
  // pl. cudaMalloc, cudaMemcpy, kernel hívás, eredmények visszamásolása, memória felszabadítás
  cudaEventRecord(stop, 0);
  ```
  A második paraméter a `cudaEventRecord` hívásokban az adatfolyamot ([CUDA stream](https://developer.download.nvidia.com/CUDA/training/StreamsAndConcurrencyWebinar.pdf)) jelöli, és a 0 érték azt jelzi, hogy az esemény a null adatfolyamhoz van rendelve.

  Az adatfolyamok (stream) olyan CUDA fogalmak, amelyek lehetővé teszik a kernel futtatását és az adatmozgatást aszinkron módon. Az adatfolyamok segítségével párhuzamosíthatjuk a műveleteket és jobban kihasználhatjuk a hardverünk párhuzamosítási képességeit.

3. A [`cudaEventSynchronize`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1g949aa42b30ae9e622f6ba0787129ff22) függvény használatával megvárjuk, amíg a stop időbélyeg rögzítése megtörténik, majd ellenőrizzük az esetleges hibákat. Például:
  ```cpp
  cudaEventSynchronize(stop);
  cudaError_t error = cudaGetLastError();
  if (error != cudaSuccess) {
      printf("CUDA hiba: %s\n", cudaGetErrorString(error));
  }
  ```

4. Végül az [`cudaEventElapsedTime`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1g40159125411db92c835edb46a0989cd6) függvény segítségével kiszámítjuk a két bélyeg között eltelt időt. Például:
  ```cpp
  float elapsedTime;
  cudaEventElapsedTime(&elapsedTime, start, stop);
  printf("A CUDA műveletek elvégzése %.2f ms telt.\n", elapsedTime);
  ```
5. A [`cudaEventDestroy`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EVENT.html#group__CUDART__EVENT_1g2cb6baa0830a1cd0bd957bfd8705045b) függvényt használjuk az általunk létrehozott `cudaEvent` objektumok felszabadítására. Például:
  ```cpp
  cudaEventDestroy(start);
  cudaEventDestroy(stop);
  ```

Az alábbi példa összefoglalja a fenti lépéseket:

```cpp
__global__ void addKernel(int *c, const int *a, const int *b, int size)
{
    int i = threadIdx.x;
    if (i < size) {
        c[i] = a[i] + b[i];
    }
}

int main()
{
    const int size = 1024;
    int a[size], b[size], c[size];

    for (int i = 0; i < size; i++) {
        a[i] = i;
        b[i] = 2 * i;
    }

    int *dev_a, *dev_b, *dev_c;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);

    cudaMalloc(&dev_a, size * sizeof(int));
    cudaMalloc(&dev_b, size * sizeof(int));
    cudaMalloc(&dev_c, size * sizeof(int));

    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);

    addKernel<<<1, size>>>(dev_c, dev_a, dev_b, size);

    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);
    
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    cudaError_t error = cudaGetLastError();
    if (error != cudaSuccess) {
        printf("CUDA hiba: %s\n", cudaGetErrorString(error));
    }
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("A CUDA művelet %.2f millió másodpercig tartott.\n", elapsedTime);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return 0;
}
```

## Időmérés CPU-n

C++-ban, egy nagyfelbontású aktuális időpontot, a
```cpp
static  std::chrono::time_point<std::chrono::high_resolution_clock> now()  noexcept;
```
[függvénnyel](https://en.cppreference.com/w/cpp/chrono/high_resolution_clock/now) kérhetjük le. A rendszer maximális időfelbontását (_ticks per second_) a `std::chrono::high_resolution_clock::period::den` érték lekérésével kaphatjuk meg.

Ha kíváncsiak vagyunk egy kódrészlet futási idejére, megmérhetjük azt két időbélyegek segítségével, melyeket lekérdezünk mindjárt a kódrészlet előtt és után. A két időbélyeg különbsége megadja az eltelt időt, a kívánt [mértékegységben](https://en.cppreference.com/w/cpp/chrono/duration/duration_cast):

![](https://i.ibb.co/F5D8brL/duration-cast.png)


### Példa időmérésre:

```cpp
#include <iostream>
#include <chrono>

using  namespace std;

int main()
{
	cout  << chrono::high_resolution_clock::period::den  << endl;
	auto start_time = chrono::high_resolution_clock::now();
	long temp =  0;
	for  (auto i =  0; i < 100000000; i++)
		temp += i;
	auto end_time = chrono::high_resolution_clock::now();
	cout  << chrono::duration_cast<chrono::seconds>(end_time - start_time).count()  <<  ":";
	cout  << chrono::duration_cast<chrono::milliseconds>(end_time - start_time).count()  <<  ":";
	cout  << chrono::duration_cast<chrono::microseconds>(end_time - start_time).count()  <<  ":";
	cout << endl;

	return  0;
}
```
Mivel ez az időmérés a rendszerórát veszi figyelembe (wall clock time), és a számítások közben más folyamatok is ütemezésre kerülhettek, a pontos eredmény érdekében érdemes többször megmérni a futási időt, az kiugró értékeket eldobni, majd a többi mérésből átlagot számolni.

A `clock_gettime` segítségével, POSIX kompatibilis rendszereken a `CLOCK_PROCESS_CPUTIME_ID` óra azonosító paraméter megadásával, folyamat szintű időmérést végezhetünk. A [https://en.cppreference.com/w/c/chrono/clock](https://en.cppreference.com/w/c/chrono/clock) található példa bemutatja a folyamat és rendszer idő közötti különbséget.

# Véletlenszám generálás CUDA-ban

A továbbiakban megismerkedünk a véletlen számok generálásának módszertanával CUDA-ban, a [`cuRAND`](https://docs.nvidia.com/cuda/curand/index.html)  könyvtár használatával.


Egy álvéletlenszám sorozat generálása egy szekvenciális folyamat, ahol a következő számot az előbbi segítségével állítjuk elő. Például a [lineáris kongruenciális generátor](https://wikihuhu.top/wiki/Lehmer_random_number_generator) esetében


```
X_{n+1} = (aX_{n} + c) mod m
```

ahol ![](https://render.githubusercontent.com/render/math?math=$a$) a szorzó , ![](https://render.githubusercontent.com/render/math?math=$c$) a növekmény és ![](https://render.githubusercontent.com/render/math?math=$m$) a modulus.

A kezdeti kifejezést magnak (angolul seed) nevezzük. Ez teszi lehetővé egy látszólag véletlenszerű szekvencia létrehozását. Minden maghoz új folytatást kapunk.

Párhuzamosan generálni pszeudo-véletlen sorozattokat [kicsit bonyolultabb](https://quick-adviser.com/is-rand--thread-safe/
), mivel egy naiv megközelítésben az `X_{n+1}` kiszámolásánál versenyhelyzet lép fel, több szál is ugyanazt  az értéket fogja generálni.


Egy egyszerű megoldás, ha  minden szálhoz külön véletlenszerű állapotot hozunk létre, amelyet az egyes szálak a saját, a többiektől független véletlenszerű számsorozat létrehozásához használnak. Minden szál különböző magból, eredeti állapotból indul ki, annak érdekében, hogy mindegyikük egy különböző álváletlenszerű sorozatot generáljon.

A [`cuRAND`](https://docs.nvidia.com/cuda/curand/index.html) könyvtár interfészt biztosít a véletlenszám-generátor állapotának inicializálásához szálanként, és ennek az állapotnak a felhasználásához véletlenszám-sorozatok generálásához. A következő lépések szükségesek ahhoz, hogy a cuRAND segítségével véletlenszerű számokat tudjunk generálni a CUDA programjainkban:

1.	A `cuRAND` fejlécek behívása.

	```cpp
	#include <curand_kernel.h>
	#include <curand.h>
	```
2.	Minden szálnak külön kell biztosítani egy eredeti magot,  különböző belső állapotot:

	-	Memória lefoglalása a szálak állapváltozóinak (`curandState`).

		```cpp
		curandState *dev_random;
		cudaMalloc((void**)&dev_random, num_threads_per_block*num_blocks*sizeof(curandState));
		```

	- Véletlenszerű állapotok inicializálása egy kernelben. Minden szál inicializálja a saját állapotát a GPU-n, a saját egyedi azonosítóját felhasználva:

		```cpp
		__global__ void gpu_random(..., curandState *states) {  
			int id = threadIdx.x + blockDim.x * blockIdx.x;  

			...  
			int seed = id; // different seed per thread         
			curand_init(seed, id, 0, &states[id]);  //  Initialize CURAND        
		 	...
		 ```

3. Véletlen számok generálása a GPU-n valamilyen eloszlás szerint (pl. `curand_uniform`), szálanként a saját inicializált állapot használatával.

	```cpp
	__global__ void gpu_random(..., curandState *states) {  
		int id = threadIdx.x + blockDim.x * blockIdx.x;  float x;         
		...  
		curand_init(seed, tid, 0, &states[id]);  //  Initialize CURAND         
		...
		for(int i = 0; i < TRIALS_PER_THREAD; i++) {   
		x = curand_uniform (&states[id]);   
		...  
	}
	```


A leggyakrabban használt eloszlások:

```cpp
// Egyenletes eloszlás
__device__ float
curand_uniform (curandState_t *state)

// Normális eloszlás
__device__ float
curand_normal (curandState_t *state)

// Log-normális eloszlás - a valószínűségi változó logaritmusa normális eloszlású.
__device__ float
curand_log_normal (curandState_t *state, float mean, float stddev)

// Poisson-eloszlás
__device__ unsigned int
curand_poisson (curandState_t *state, double lambda)
```

Ezek a függvények elérhetőek dupla pontosságú lebegőpontos (`double`) számábrázolással is.

## [Példa](https://github.com/deeperlearning/professional-cuda-c-programming/blob/master/examples/chapter08/rand-kernel.cu)

```cpp
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand_kernel.h>


int threads_per_block = 256;
int blocks_per_grid = 30;


/*
 * device_api_kernel uses the cuRAND device API to generate random numbers
 * on-the-fly on the GPU, and then performs some dummy computation using them.
 */
__global__ void device_api_kernel(curandState *states, float *out, int N)
{
    int i;
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int nthreads = gridDim.x * blockDim.x;
    curandState *state = states + tid;

    curand_init(9384, tid, 0, state);

    for (i = tid; i < N; i += nthreads)
    {
        float rand = curand_uniform(state);
        rand = rand * 2;
        out[i] = rand;
    }
}


/*
 * use_device_api is an examples usage of the cuRAND device API to use the GPU
 * to generate random values on the fly from inside a CUDA kernel.
 */
void use_device_api(int N)
{
    int i;
    static curandState *states = NULL;
    float *dOut, *hOut;

    /*
     * Allocate device memory to store the output and cuRAND device state
     * objects (which are analogous to handles, but on the GPU).
     */
    cudaMalloc((void **)&dOut, sizeof(float) * N);
    cudaMalloc((void **)&states, sizeof(curandState) *
                threads_per_block * blocks_per_grid);
    hOut = (float *)malloc(sizeof(float) * N);

    // Execute a kernel that generates and consumes its own random numbers
    device_api_kernel<<<blocks_per_grid, threads_per_block>>>(states, dOut, N);

    // Retrieve the results
    cudaMemcpy(hOut, dOut, sizeof(float) * N, cudaMemcpyDeviceToHost);

    printf("Sampling of output from device API:\n");

    for (i = 0; i < 10; i++)
    {
        printf("%2.4f\n", hOut[i]);
    }

    printf("...\n");

    free(hOut);
    cudaFree(dOut);
    cudaFree(states);
}

int main(int argc, char **argv)
{
    int N = 8388608;

    use_device_api(N);

    return 0;
}
```



#CUDA profilozása az NVIDIA Profiler segítségével

A CUDA-programok hatékonyságának elemzéséhez gyakran nem elég csupán a kernel futási időt mérni, mivel ez nem ad részletes információt a **memóriahozzáférésekről, regiszterhasználatról, párhuzamos végrehajtás hatékonyságáról és az adatmozgatásokról**. Az [NVIDIA Profiler eszközei](https://developer.nvidia.com/performance-analysis-tools), például **nvprof, Nsight Systems és Nsight Compute**, lehetőséget adnak mélyebb teljesítményprofilozásra.  

**Mivel más vagy több  az NVIDIA Profiler, mint egy egyszerű időmérés?**  
- Megmutatja, hogy a kernel futásán belül hol vannak szűk keresztmetszetek.  
- Elemzi a **memóriahasználatot** (globális, megosztott memória, L2 cache kihasználtság).  
- Vizsgálja a **számítási kihasználtságot** (SM-kihasználtság, párhuzamos végrehajtás).  
- Részletes jelentést nyújt az egyes CUDA API hívásokról és memóriamásolásokról.  

Az [nvprof](https://docs.csc.fi/computing/nvprof/) eszköz egy parancssori profiler, nem igényel GUI-t, ezért Google Colab környezetben is könnyen tudjuk használni.


Az nvprof-nak csak egyszerűen át kell adnunk a CUDA programunkat mint paraméter:

```bash
nvprof ./cuda_program
```

Ez majd automatikusan rögzíti az összes CUDA API hívást, kernel futtatást és memóriaátvitelt, majd egy összegző jelentést készít.

---

## Példa -  CUDA program profilozása
Vizsgáljuk meg egy egyszerű CUDA-kernel futását és memóriahasználatát az nvprof segítségével.


In [ ]:
%%writefile profiletest.cu

#include <cuda.h>
#include <curand_kernel.h>
#include <stdio.h>

__global__ void vector_add(float *a, float *b, float *c, int n, unsigned long long seed) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;

    // Véletlenszám-generátor inicializálása minden szálnál
    curandState state;
    curand_init(seed, i, 0, &state);

    if (i < n) {
        float noise = curand_uniform(&state) * 0.1f - 0.05f; // Zaj [-0.05, 0.05] intervallumban
        c[i] = a[i] + b[i] + noise;
    }
}

int main() {
    int n = 1000000;
    size_t size = n * sizeof(float);

    // Memória foglalás a hoston
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);

    // Inicializálás
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    // Memória foglalás a GPU-n
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Adatok másolása a GPU-ra
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Kernel indítása
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    vector_add<<<numBlocks, blockSize>>>(d_a, d_b, d_c, n, 1234ULL);

    // Eredmények visszamásolása
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Memória felszabadítás
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    free(h_a); free(h_b); free(h_c);

    return 0;
}



Writing profiletest.cu


In [ ]:
!nvcc -arch=sm_75 -o vector_add_noise profiletest.cu

In [ ]:
!./vector_add_noise

In [ ]:
!nvprof --version

nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2024 NVIDIA Corporation
Release version 12.5.82 (21)


In [ ]:
!nvidia-smi

Tue Mar 25 15:40:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvprof ./vector_add_noise

==12986== NVPROF is profiling process 12986, command: ./vector_add_noise
==12986== Profiling application: ./vector_add_noise
==12986== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   93.16%  41.018ms         1  41.018ms  41.018ms  41.018ms  vector_add(float*, float*, float*, int, __int64)
                    3.72%  1.6385ms         1  1.6385ms  1.6385ms  1.6385ms  [CUDA memcpy DtoH]
                    3.12%  1.3719ms         2  685.95us  684.72us  687.19us  [CUDA memcpy HtoD]
      API calls:   71.18%  119.69ms         3  39.897ms  64.851us  119.55ms  cudaMalloc
                   27.52%  46.274ms         3  15.425ms  906.25us  43.921ms  cudaMemcpy
                    0.60%  1.0016ms         1  1.0016ms  1.0016ms  1.0016ms  cuDeviceGetPCIBusId
                    0.32%  542.03us         3  180.68us  127.45us  209.25us  cudaFree
                    0.29%  484.67us         1  484.67us  484.67us  484.67us  cudaLaunchK

**Mit tudunk meg ebből?**
- A **vector_add kernel** a teljes futási idő **92.15%-át** teszi ki.
- Az **adatmásolás** (Host → Device és Device → Host) mennyi időt vesz igénybe (~4.47% + 3.38%).
- Milyen API hívások hajtodtak végre.


## Részletesebb Elemzés
Az nvprof további hasznos lehetőségeket biztosít. A GPU-műveletek részletes megjelenítése hasnzáljuk:


In [ ]:
!nvprof --print-gpu-trace ./vector_add_noise

==13065== NVPROF is profiling process 13065, command: ./vector_add_noise
==13065== Profiling application: ./vector_add_noise
==13065== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
167.49ms  759.38us                    -               -         -         -         -  3.8147MB  4.9057GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
168.49ms  746.42us                    -               -         -         -         -  3.8147MB  4.9909GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
169.82ms  41.030ms           (3907 1 1)       (256 1 1)        61        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  vector_add(float*, float*, float*, int, __int64) [130]
210.85ms  1.9315ms                    -               -         -    

# New section

```bash
nvprof --print-gpu-trace ./vector_add
```

A profilozás az alkalmazás futtatásának különböző aspektusait méri, mint például a memória másolásokat, kernel futtatásokat, és a GPU erőforrások használatát. A profilozás minden CUDA művelet pontos időbélyegét megmutatja. Pontosan mit láthatunk?


### Értelmezés

1. CUDA Memcpy HtoD (Host-to-Device Memory Copy)
- **`243.29ms`** és **`244.22ms`**: Ez az időpont, amikor a memória másolás elkezdődik és befejeződik. A másolás a host (CPU) és a device (GPU) között történik.
- **`696.81us` és `707.98us`**: A memória másolás (Host to Device) ideje, tehát mennyi időbe telt az adatokat átmásolni a CPU-ról a GPU-ra.
- **`3.8147MB`**: A memória másolás mérete. Ez azt jelenti, hogy az adatok mérete körülbelül 3.8 MB volt, amit átvittünk.
- **`5.3462GB/s` és `5.2619GB/s`**: A memória másolás sebessége, azaz hány GB adatot másoltunk másodpercenként.

A **`SrcMemType`** és **`DstMemType`** az adatok forrás- és célmemóriájának típusát jelzi.
- **`Pageable`**: Ez azt jelenti, hogy a memóriát nem tartja fixen a rendszer, így a memória elérhetősége rugalmas.
- **`Device`**: A célmemória a GPU memóriája.


2. **CUDA Memcpy DtoH (Device-to-Host Memory Copy)**
- **`286.35ms`** és **`1.6342ms`**: A másolás befejeződése és az időtartam. Ez az adat visszamásolását jelenti a GPU-ról a CPU-ra.
- **`3.8147MB`**: A visszamásolt adat mennyisége.
- **`2.2796GB/s`**: Az adatok átvitelének sebessége a GPU-ról a CPU-ra.


3. **Kernel futás: `vector_add`**
- **`245.41ms`**: A kernel végrehajtásának időtartama, ami a `vector_add` nevű függvény futtatása.
- **`40.934ms`**: Ez az idő, amely alatt a kernel ténylegesen végrehajtódott.
- **`(3907 1 1)`**: A **grid mérete** (háromdimenziós), ami azt jelzi, hogy hány blokkot futtatott a kernel: 3907 blokkot.
- **`(256 1 1)`**: A **block mérete** (háromdimenziós), ami azt jelzi, hogy hány szálat futtatott egy blokkban: 256 szál.
- **`61`**: A kernel által használt **regiszterek** száma szálanként. A regiszterek a CPU/GPU gyors memóriája, amely a szálak gyors adatkezeléséhez szükségesek.
- **`0B`**: **Static shared memory (SSMem)** és **Dynamic shared memory (DSMem)**: Ez a memória, amelyet a blokk szálai osztanak meg egymással. Most nem lett használva sem statikus, sem dinamikus memória.
  
- A kernel neve: **`vector_add(float*, float*, float*, int, __int64)`**, ami azt jelzi, hogy egy vektorműveletet hajtott végre, ahol három lebegőpontos számokból álló vektort adtak össze.



### Következtetés

- Az adatátvitel (Host-to-Device és Device-to-Host) viszonylag gyors volt, 5-5 GB/s sebességgel.
- A **`vector_add`** kernel futása 245.41ms-ig tartott, és az átlagos végrehajtási idő 40.934ms volt. Az adatokat 3907 blokkban és 256 szálat használva dolgozták fel.
- A regiszterek száma szálanként 61, ami azt jelzi, hogy nem használtak túl sok regisztert a kernelben. A kernel nem használt megosztott memóriát (sem statikus, sem dinamikus).

Mint látjuk, a profilozás segít abban, hogy áttekinthessük a program GPU erőforrás-használatát, és láthatjuk, hogy a memória másolás és a kernel végrehajtás hogyan befolyásolja a program teljesítményét.




## Memóriahasználat Profilozása

In [ ]:
!nvprof --print-gpu-summary ./vector_add_noise

==4856== NVPROF is profiling process 4856, command: ./vector_add_noise
==4856== Profiling application: ./vector_add_noise
==4856== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   92.87%  40.918ms         1  40.918ms  40.918ms  40.918ms  vector_add(float*, float*, float*, int, __int64)
                    3.68%  1.6196ms         1  1.6196ms  1.6196ms  1.6196ms  [CUDA memcpy DtoH]
                    3.45%  1.5220ms         2  761.00us  743.66us  778.35us  [CUDA memcpy HtoD]


Az **`nvprof --print-gpu-summary`** parancs összegző profilozási adatokat adott vissza a CUDA programról, beleértve a GPU aktivitások időarányos megoszlását, valamint a különböző kernel-ek és memória műveletek végrehajtásának időtartamát.

A következő adatokat láthatjuk az eredményben:

1. GPU aktivitások:
- **`92.87%`**: A **GPU aktivitások összes idejének 92.87%-át** a `vector_add` kernel végrehajtása tette ki. Ez azt jelzi, hogy a program fő részét a kernel futtatása képezte.
  - **`40.918ms`**: A `vector_add` kernel teljes futási ideje.
  - **`1`**: A kernel pontosan **1 alkalommal** futott.
  - **`40.918ms`**: Az egyes futások időtartama, ami azonos minden egyes kernel végrehajtás esetén (mivel csak egyszer futott).

2. **[CUDA memcpy DtoH] (Device-to-Host Memory Copy):
- **`3.68%`**: A memória másolás a GPU-ról a CPU-ra (Device-to-Host) **3.68%-ban** tette ki a teljes időt.
  - **`1.6196ms`**: Ez a másolás teljes időtartama.
  - **`1`**: A memória másolás pontosan **1 alkalommal** történt.
  - **`1.6196ms`**: Az egyes memória másolások időtartama (mivel csak egyszer történt másolás).

3. **[CUDA memcpy HtoD] (Host-to-Device Memory Copy):
- **`3.45%`**: A memória másolás a CPU-ról a GPU-ra (Host-to-Device) **3.45%-ban** tette ki a teljes időt.
  - **`1.5220ms`**: A memória másolás teljes időtartama (összesen két másolás történt).
  - **`2`**: A memória másolás **2 alkalommal** történt.
  - **`761.00us`**: Az egyes másolások átlagos időtartama (768us és 743us között ingadozott).
  - **`743.66us`** és **`778.35us`**: A másolások közötti időeltérés, tehát az időintervallumok minimális és maximális értékei.

Összegzés:
- **A `vector_add` kernel** a legnagyobb részét tette ki a program futásának, és **40.918 ms**-ot igényelt.
- **Memória másolás**:
  - A **GPU-ról a CPU-ra történő másolás** (`[CUDA memcpy DtoH]`) körülbelül **1.6196 ms**-ot vett igénybe.
  - A **CPU-ról a GPU-ra történő másolás** (`[CUDA memcpy HtoD]`) összesen **1.5220 ms** időt vett igénybe, amelyet két különböző másolás hajtott végre.

Itt is látjuk, a legnagyobb időt a **kernel végrehajtása** tette ki, míg a memória másolás (GPU és CPU között) viszonylag kis időt vett igénybe az összes futási időhöz képest.


---

## Általános optimalizálási tippek
- Memóriahasználat csökkentése: Ha a memória másolás időigényes, próbáljunk meg kevesebb adatot mozgatni.  
- Regiszterhasználat figyelése: Ha túl sok a regiszter, kevesebb thread tud futni párhuzamosan.  
- Optimalizált grid/block méretek: Ha a kihasználtság alacsony, próbáljunk meg más **blockSize** értékeket.  
- Ha az L2 cache hit rate alacsony, érdemes **megosztott memóriát (shared memory)** használni a teljesítmény növelése érdekében.


## nvprof összegzés
| **Parancs** | **Mit csinál?** |
|-------------|----------------|
| `nvprof ./program` | Alapvető profilozás, futási idők elemzése |
| `nvprof --print-gpu-trace ./program` | Részletes GPU műveletek megjelenítése |
| `nvprof --print-gpu-summary ./program` | Memóriahasználat és kihasználtság elemzése |

Az nvprof egy egyszerű, de erőteljes parancssori eszköz a CUDA-programok teljesítményének mérésére. Ha mélyebb elemzést szeretnénk, az **Nsight Systems és Nsight Compute** további részleteket adhat.


# Feladatok

1. Írjunk egy CUDA programot, amely véletlenszerű pontokat generál az egységnégyzetben \([0,1] \times [0,1]\), majd kiszámolja a pontok távolságát az origótól.  
  Lépések:
  - Minden CUDA szál generáljon egy véletlen \( (x, y) \) pontot a \([0,1]\) tartományban.  
  - Számoljuk ki a pontok távolságát az origótól:  
   $d = \sqrt{x^2 + y^2}$
  - Tároljuk az eredményeket egy tömbben.  
  - Másoljuk vissza az adatokat a CPU-ra, és számoljuk ki az átlagos távolságot.  

  Az átlagos távolságnak közel kell lennie az elméleti értékhez:  $E[d] \approx 0.521405$

2. Írjunk egy szekvenciális majd egy CUDA programot mely Monte Carlo módszer segítségével, ![](https://render.githubusercontent.com/render/math?math=$n$) véletlenszerű pontot generálva, megközelítően kiszámítja a ![](https://render.githubusercontent.com/render/math?math=$\pi$) értékét. Az ![](https://render.githubusercontent.com/render/math?math=$n$) a program paramétere.

  A módszer leírása:
    
  - [http://nagysandor.eu/physlet/applets/iter1.html](http://nagysandor.eu/physlet/applets/iter1.html)

  - [http://www.tldp.org/HOWTO/Parallel-Processing-HOWTO-2.html](http://www.tldp.org/HOWTO/Parallel-Processing-HOWTO-2.html)

3. A tanult CUDA időmérést felhasználva számoljuk ki CUDA-programjaink gyorsulását. Ehhez természetesen szükség van a szekvenciális verziókra is, amelyek a hoston futnak.  

  A hoston az időmérést a `chrono::high_resolution_clock` segítségével végezzük, míg a CUDA-kernélek esetében a `cudaEventCreate` és `cudaEventRecord` függvényeket használjuk.

4. Profilozzuk a megírt programokat. Vizsgáljuk meg a kernelek futási idejét, a regiszterek és memóriahasználatot, majd az észrevételeket egy rövid beszámolóban rögzítsük (ez lehet külön fájl vagy egy új "text" cella a notebook-ban).


In [6]:
%%writefile first_exercise.cu

#include <stdio.h>
#include <curand_kernel.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

// Kernel: minden szál generál egy véletlen pontot és kiszámolja a távolságot az origótól.
__global__ void kernel(float *distances, unsigned int seed, int n) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < n) {
        // Inicializáljuk a curand állapotot minden szálban
        curandState state;
        curand_init(seed, idx, 0, &state);

        // Véletlen számok [0,1]-ben
        float x = curand_uniform(&state);
        float y = curand_uniform(&state);

        // Távolság az origótól: sqrt(x*x + y*y)
        float d = sqrtf(x * x + y * y);
        distances[idx] = d;
    }
}

int main() {
    const int n = 1000000; // Pontok száma
    size_t size = n * sizeof(float);

    // Host memóriában tárolt eredmény tömb
    float *h_distances = (float *) malloc(size);

    // Device memóriára foglalás
    float *d_distances;
    cudaMalloc((void **) &d_distances, size);

    // Kernel konfiguráció: 256 szál blokkanként
    int threadsPerBlock = 256;
    int blocks = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Kernel indítása; a seed értéke a rendszeridő alapján kerül beállításra
    kernel<<<blocks, threadsPerBlock>>>(d_distances, time(NULL), n);
    cudaDeviceSynchronize();

    // Adatok átvitele a GPU-ról a CPU-ra
    cudaMemcpy(h_distances, d_distances, size, cudaMemcpyDeviceToHost);

    // Átlagos távolság kiszámolása a CPU-n
    double sum = 0.0;
    for (int i = 0; i < n; i++) {
        sum += h_distances[i];
    }
    double avg = sum / n;
    printf("Átlagos távolság: %f\n", avg);

    // Memória felszabadítása
    cudaFree(d_distances);
    free(h_distances);

    return 0;
}

Overwriting first_exercise.cu


In [7]:
!nvcc -arch=sm_75 -o first_exercise_out first_exercise.cu

In [8]:
!./first_exercise_out

Átlagos távolság: 0.765073


In [13]:
%%writefile second_exercise.cu


#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

int main(int argc, char *argv[]) {
    if (argc != 2) {
        printf("Használat: %s pontok_száma\n", argv[0]);
        return 1;
    }

    long long num_points = atoll(argv[1]);
    long long count_inside = 0;

    // Véletlen generátor inicializálása
    srand(time(NULL));

    // Monte Carlo: generáljunk pontokat az [0,1]x[0,1] egységnégyzetben
    for (long long i = 0; i < num_points; i++) {
        double x = (double)rand() / RAND_MAX;
        double y = (double)rand() / RAND_MAX;
        // Ha a pont az egységsugarú körön belül van
        if (x*x + y*y <= 1.0) {
            count_inside++;
        }
    }

    // π közelítése: a kör területe ¼ * π, így π ≈ 4 * (belső pontok száma) / (összes pontok száma)
    double pi = 4.0 * count_inside / num_points;
    printf("Közelített π érték: %f\n", pi);

    return 0;
}


Overwriting second_exercise.cu


In [14]:
!nvcc -arch=sm_75 -o second_exercise_out second_exercise.cu

In [21]:
!./second_exercise_out 10000000

Közelített π érték: 3.141109


In [16]:
%%writefile second_exercise.cu

#include <stdio.h>
#include <curand_kernel.h>
#include <cuda_runtime.h>
#include <time.h>

// CUDA kernel, mely minden szál a saját részfeladatát végzi.
// Az eredmény (a kör belsejében lévő pontok száma) globális változóba kerül atomikusan összegzésre.
__global__ void monteCarloPi(unsigned long long *count, int num_points, unsigned int seed) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    unsigned long long local_count = 0;
    curandState state;

    // Minden szál inicializálja a saját curand állapotát
    curand_init(seed, idx, 0, &state);

    // Grid-stride ciklussal több iterációt is lefutattunk, ha a szálak száma kevesebb, mint a pontok száma.
    for (int i = idx; i < num_points; i += blockDim.x * gridDim.x) {
        float x = curand_uniform(&state);
        float y = curand_uniform(&state);
        if (x * x + y * y <= 1.0f) {
            local_count++;
        }
    }
    // Az atomikus összeadás segítségével összegezzük az egyes szálak eredményét.
    atomicAdd(count, local_count);
}

int main(int argc, char *argv[]) {
    if (argc != 2) {
        printf("Használat: %s pontok_száma\n", argv[0]);
        return 1;
    }

    int num_points = atoi(argv[1]);
    unsigned long long h_count = 0;
    unsigned long long *d_count;

    // Memória foglalása a GPU-n a pontok számának összegzésére
    cudaMalloc((void**)&d_count, sizeof(unsigned long long));
    cudaMemcpy(d_count, &h_count, sizeof(unsigned long long), cudaMemcpyHostToDevice);

    // CUDA kernel futtatása: blokkonként 256 szál, a blokkok számát úgy számoljuk, hogy lefedjük az összes pontot
    int threadsPerBlock = 256;
    int blocks = (num_points + threadsPerBlock - 1) / threadsPerBlock;

    // Véletlen seed a rendszeridő alapján
    unsigned int seed = time(NULL);
    monteCarloPi<<<blocks, threadsPerBlock>>>(d_count, num_points, seed);
    cudaDeviceSynchronize();

    // Az eredmény átmásolása a CPU memóriába
    cudaMemcpy(&h_count, d_count, sizeof(unsigned long long), cudaMemcpyDeviceToHost);

    // π közelítése: 4 * (belső pontok száma) / (összes pontok száma)
    double pi = 4.0 * h_count / num_points;
    printf("Közelített π érték: %f\n", pi);

    // GPU memória felszabadítása
    cudaFree(d_count);
    return 0;
}


Overwriting second_exercise.cu


In [17]:
!nvcc -arch=sm_75 -o second_exercise_out second_exercise.cu

In [23]:
!./second_exercise_out 10000000

Közelített π érték: 3.141168
